In [ ]:
from pathlib import Path

import pytz

from post_processing.utils.fpod_utils import csv_folder, pod2aplose, actual_data, meta_cut_aplose, feeding_buzz, txt_folder, add_utc
from post_processing.utils.core_utils import json2df

### Load data
🐬 = input to modify \
Import your raw FPOD or CPOD data. All files for one site must be stored in the same folder and identified by their respective phases. \
You also need to import your metadata file.

In [ ]:
pod_files = Path(r"U:\CA4") #Path to your data folder. 🐬
path = csv_folder(pod_files) #Process all your POD.csv files.

fb_files = Path(r"U:\fb_CA4")  #Path to your click details folder. 🐬
json = Path(r"C:\Users\fouinel\Downloads\deployment_calais.json") #Path to your metadata file. 🐬

print(path.head())
df_0 = path.dropna()

metadatax = json2df(json_path=json)
metadatax["deploy.name"] = (metadatax["site.name"].astype(str) + "_" +  metadatax["campaign.name"].astype(str))

In [ ]:
d_beg_end = actual_data(df_0, metadatax) #Extract the beginning and end of recording for every phase.

In [ ]:
df_1 = df_0[df_0["DPM"] !="0" ] #Remove the 0 to lighten the APLOSE file.

### APLOSE format
#### *C-POD*
Use cpod2aplose if you are managing C-POD data.

In [ ]:
df_aplose = pod2aplose(df_1, pytz.utc, "CA4", "Marsouin", "CPOD") #Precise site name, species and instrument. 🐬
df_aplose["deploy.name"] = df_aplose["filename"]
print(df_aplose.head())

### Clean your dataset
Remove useless lines, recorder outside the instrument submersion. Export your file to the aplose format. You can change the name of the file to match the project you are working on.

In [ ]:
cleared = meta_cut_aplose(df_aplose, metadatax)

### Feeding buzzes processing
Use "Dauphin", Marsouin" or "Commerson" to get different ICI processing.

In [ ]:
fb_all = txt_folder(fb_files) #Read all your FB.txt files.
fb_all = feeding_buzz(fb_all, "Marsouin") #Categorize the minutes (positive or not to FB detection). 🐬
add_utc(fb_all, ["start_datetime"], "min")

In [ ]:
dpm_fb = cleared.merge(fb_all, on=["start_datetime"], how="left") #Merge DPM and FB dataframes

### Export

In [ ]:
d_beg_end.to_csv(r"U:\Deb_Fin_CA4.csv", index=False) #Export the new file. 🐬
dpm_fb.to_csv(r"U:\APLOSE_CA4_pos.csv", index=False) #Name your file. 🐬